In [1]:
import json 
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize #package for flattening json in pandas df
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
df = pd.read_json(r'/opt/data/articles_list_Oct12_2020.json', orient='columns')

#### Importing Source-Lean-Flags.csv

In [11]:
df_src_flags=pd.read_csv('source-lean-flags.csv')

In [12]:
df_src_flags.describe()

,bias_score,factually_questionable_flag,propaganda_flag,conspiracy_flag,hate_group_flag,satire_flag
count,2723.000000,3050.000000,3050.000000,3050.000000,3050.000000,3050.000000
mean,0.371282,0.273443,0.028852,0.096066,0.022623,0.038033
std,1.762916,0.445799,0.167419,0.294729,0.148723,0.191307
min,-3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
df_src_flags.count()

domain                         3050
bias_score                     2723
factually_questionable_flag    3050
propaganda_flag                3050
conspiracy_flag                3050
hate_group_flag                3050
satire_flag                    3050
dtype: int64

In [15]:
df_src_flags.shape

(3050, 7)

## Basic EDA

In [3]:
df.describe()

,_score
count,60374.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [4]:
df.count()

_index     60374
_type      60374
_id        60374
_score     60374
_source    60374
dtype: int64

In [16]:
df.shape

(60374, 5)

In [6]:
#print 1st column 1st row
df.iloc[0] # this is same as df[[0]].values[0][0]

_index                                              articles
_type                                                   _doc
_id                                     jzHWHnUBlmHqFksVvSCK
_score                                                     1
_source    {'url': 'http://actionnewsjax.com/news/trendin...
Name: 0, dtype: object

In [7]:
df_clean= df.drop(['_id','_score', '_type'], axis=1)

In [30]:
df_clean.shape

(60374, 2)

In [18]:
df_clean.head(5)

,_index,_source
0,articles,{'url': 'http://actionnewsjax.com/news/trendin...
1,articles,{'url': 'https://www.13newsnow.com/article/new...
2,articles,{'url': 'http://digg.com/digg-picks/link/kindl...
3,articles,{'url': 'http://sports.mynorthwest.com/1199798...
4,articles,{'url': 'https://www.aier.org/article/lockdown...


In [17]:
df_clean.iloc[0]

_index                                              articles
_source    {'url': 'http://actionnewsjax.com/news/trendin...
Name: 0, dtype: object

In [19]:
df.iloc[0]

_index                                              articles
_type                                                   _doc
_id                                     jzHWHnUBlmHqFksVvSCK
_score                                                     1
_source    {'url': 'http://actionnewsjax.com/news/trendin...
Name: 0, dtype: object

In [20]:
df.describe()

,_score
count,60374.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [21]:
df.dtypes

_index     object
_type      object
_id        object
_score      int64
_source    object
dtype: object

In [22]:
#Get value within a Series
df.loc[0].at['_type']

'_doc'

In [44]:
#Get value within a Series
df.loc[0].at['_source']

{'url': 'http://actionnewsjax.com/news/trending/amy-coney-barrett-hearing-day-2-questioning-begins-livestream-live-updates/MSRUGTDZVRFO3M7PXIQC6UY45U/',
 'extracted': {'title': 'Amy Coney Barrett hearing Day 2: Questioning begins; livestream, live updates',
  'date': None,
  'authors': ['Debbie Lord',
   'Cox Media Group National Content Desk',
   'Updated',
   'October',
   '- Pm',
   'Kelli Dugan',
   'Crystal Bonvillian',
   'Brianna Chambers',
   'Theresa Seiger'],
  'text': '“His judicial philosophy was straightforward: A judge must apply the law as written, not as the judge wishes it were. Sometimes that approach meant reaching results that he did not like. But as he put it in one of his best-known opinions, that is what it means to say we have a government of laws, not of men.”'},
 'metadata': {'keywords': ['amy coney barrett',
   'amy coney barrett hearing',
   'what time doe the amy coney barrett hearing start',
   'what channel is the amy coney barrett hearing on',
   'livest

In [114]:
df_src=df.drop(['_index','_id','_score', '_type'], axis=1)

In [115]:
df_src.dtypes

_source    object
dtype: object

In [116]:
type(df_src)

pandas.core.frame.DataFrame

In [117]:
%%time
df_src_norm = json_normalize(df_src['_source'])

CPU times: user 51.5 s, sys: 661 ms, total: 52.2 s
Wall time: 52.1 s


In [118]:
df_src_norm.shape

(60374, 2481)

In [113]:
# List all column names
#list(df_src_norm.columns) 
#print(df_src_norm.columns.tolist())

In [45]:
df_src_norm.loc[0]

url                                                                   http://actionnewsjax.com/news/trending/amy-con...
extracted.title                                                       Amy Coney Barrett hearing Day 2: Questioning b...
extracted.date                                                                                                     None
extracted.authors                                                     [Debbie Lord, Cox Media Group National Content...
extracted.text                                                        “His judicial philosophy was straightforward: ...
                                                                                            ...                        
metadata.dump.webit_cover_date                                                                                      NaN
metadata.dump.burntime                                                                                              NaN
metadata.dump.Everything we know about A

In [102]:
#domain_cols = [col for col in df_src_norm.columns if 'domain' in col]
#print(list(df.columns))
#print(domain_cols)
#type(domain_cols)
#df_domain = df_src_norm[domain_cols]

['metadata.dump.twitter.domain', 'metadata.dump.facebook-domain-verification', 'metadata.dump.p.domain_verify', 'metadata.dump.dailymotion-domain-verification', 'metadata.dump.og.domain', 'metadata.dump.com.silverpop.brandeddomains', 'metadata.dump.update-cart-domain', 'metadata.dump.preview-new-proration-domain', 'metadata.dump.preview-change-proration-domain', 'metadata.dump.sso-api-domain', 'metadata.dump.revv-api-domain', 'metadata.dump.analytics.site_domain', 'metadata.dump.globalsign-domain-verification', 'metadata.dump.zoom-domain-verification', 'metadata.dump.cna.subdomain_id', 'metadata.dump.rand-domains', 'metadata.dump.domain', 'metadata.dump.“p.domain_verify”', 'metadata.dump.dfp.domain']


In [103]:
#df_domain.dropna(how='all')

,metadata.dump.twitter.domain,metadata.dump.facebook-domain-verification,metadata.dump.p.domain_verify,metadata.dump.dailymotion-domain-verification,metadata.dump.og.domain,metadata.dump.com.silverpop.brandeddomains,metadata.dump.update-cart-domain,metadata.dump.preview-new-proration-domain,metadata.dump.preview-change-proration-domain,metadata.dump.sso-api-domain,metadata.dump.revv-api-domain,metadata.dump.analytics.site_domain,metadata.dump.globalsign-domain-verification,metadata.dump.zoom-domain-verification,metadata.dump.cna.subdomain_id,metadata.dump.rand-domains,metadata.dump.domain,metadata.dump.“p.domain_verify”,metadata.dump.dfp.domain
1,www.13newsnow.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,www.aier.org,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,rte.ie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,it3qhk73o3gnhiw5gnatg2k4vs32ks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,NaN,NaN,69735562919053270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60358,NaN,NaN,d01ad36f4033677a2e4883eee1a13af1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60360,NaN,NaN,d01ad36f4033677a2e4883eee1a13af1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60367,NaN,NaN,755183be01bfc739a9cac6673f1f2c1b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60371,NaN,NaN,9f8dcf91c6164489adc6f0a51f7b07bf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
#df_domain.shape

(60374, 19)

In [105]:
#df_domain_t=df_domain['metadata.dump.twitter.domain']

In [106]:
#df_domain_t =df_domain_t.dropna()

In [107]:
#df_domain_t.shape

(3499,)

### Only 3499 records in the original dataset have any domain names out of 60,000+ records. So, mapping up source-lean-flags.csv data into the original dataset will only give us much smaller data to play with


In [108]:
#df_domain_t.head(100)

1       www.13newsnow.com
4            www.aier.org
5                  rte.ie
18                 rte.ie
36                 rte.ie
              ...        
1501    www.thestreet.com
1511          www.bbc.com
1526               rte.ie
1542            Firstpost
1546         bitchute.com
Name: metadata.dump.twitter.domain, Length: 100, dtype: object

In [119]:
df_src_norm.head(3)

,url,extracted.title,extracted.date,extracted.authors,extracted.text,metadata.keywords,metadata.description,metadata.language,metadata.url,metadata.dump.distribution,...,"metadata.dump.The Echelon exercise bike is $100 off, down to $499, right now at Walmart — and it comes with a free 6-month membership",metadata.dump.DC.source,metadata.dump.DC.date.published,metadata.dump.webit_document_id,metadata.dump.webit_document_name,metadata.dump.webit_cover_date,metadata.dump.burntime,metadata.dump.Everything we know about Ariana Grande's sixth album,metadata.dump.article_references,metadata.dump.epdf_available
0,http://actionnewsjax.com/news/trending/amy-con...,Amy Coney Barrett hearing Day 2: Questioning b...,None,"[Debbie Lord, Cox Media Group National Content...",“His judicial philosophy was straightforward: ...,"[amy coney barrett, amy coney barrett hearing,...",Judge Amy Coney Barrett will face questions fr...,en,https://www.actionnewsjax.com/news/trending/am...,global,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.13newsnow.com/article/news/nation-...,"Vanessa Bryant wishes Kobe, Gigi could've seen...",2020-10-12T19:24:52,"[Author, Douglas Jones, Tegna]",Vanessa Bryant wrote that she wished Kobe and ...,[],After the Los Angeles Lakers won the NBA champ...,en,https://www.13newsnow.com/article/news/nation-...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://digg.com/digg-picks/link/kindles-video-...,"Kindles, Video Doorbells And More Are On Sale ...",None,[Digg Editors],👋 Welcome to Digg\n\nThanks for creating an ac...,[],"<p>If you're a Prime member, Amazon has a nice...",en,https://digg.com/digg-picks/link/kindles-video...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
df_src_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60374 entries, 0 to 60373
Columns: 2481 entries, url to metadata.dump.epdf_available
dtypes: bool(2), float64(251), object(2228)
memory usage: 1.1+ GB


In [111]:
#for col in df_src_norm.columns:
#    if 'metadata' in col:
#        del df_src_norm[col]

In [121]:
df_src_norm.shape

(60374, 2481)

In [122]:
df_src_norm_limited=df_src_norm.filter(items=['url', 'extracted.title', 'extracted.date', 'extracted.authors', 'extracted.text', 'source.url', 'source.sitename', 'flags.in_graph', 'flags.has_nlp', 'context.scraper', 'context.datetime', 'flags.skipped_graph', 'metadata.dump.twitter.domain'])

In [123]:
df_src_norm_limited.head(10)

,url,extracted.title,extracted.date,extracted.authors,extracted.text,source.url,source.sitename,flags.in_graph,flags.has_nlp,context.scraper,context.datetime,flags.skipped_graph,metadata.dump.twitter.domain
0,http://actionnewsjax.com/news/trending/amy-con...,Amy Coney Barrett hearing Day 2: Questioning b...,None,"[Debbie Lord, Cox Media Group National Content...",“His judicial philosophy was straightforward: ...,http://actionnewsjax.com,WJAX,False,False,newspaperbasic,2020-10-12T22:03:11.021791+00:00,NaN,NaN
1,https://www.13newsnow.com/article/news/nation-...,"Vanessa Bryant wishes Kobe, Gigi could've seen...",2020-10-12T19:24:52,"[Author, Douglas Jones, Tegna]",Vanessa Bryant wrote that she wished Kobe and ...,https://www.13newsnow.com,13newsnow.com,False,False,newspaperbasic,2020-10-12T22:03:11.185397+00:00,NaN,www.13newsnow.com
2,http://digg.com/digg-picks/link/kindles-video-...,"Kindles, Video Doorbells And More Are On Sale ...",None,[Digg Editors],👋 Welcome to Digg\n\nThanks for creating an ac...,http://digg.com,,False,False,newspaperbasic,2020-10-12T22:03:11.585230+00:00,NaN,NaN
3,http://sports.mynorthwest.com/1199798/clayton-...,Seahawks Observations: John Clayton on 5-0 Haw...,2020-10-12T21:52:23+00:00,"[John Clayton, Share, October, At Pm]",The Seahawks’ 27-26 victory over the Minnesota...,http://sports.mynorthwest.com,sports.MyNorthwest.com,False,False,newspaperbasic,2020-10-12T22:03:12.004826+00:00,NaN,NaN
4,https://www.aier.org/article/lockdowns-have-ki...,Lockdowns Have Killed What’s Left of the Unite...,2020-10-12T12:30:15+00:00,"[Peter C. Earle, October, Peter C. Earle Is An...",,https://www.aier.org,AIER,False,False,newspaperbasic,2020-10-12T22:03:12.084973+00:00,NaN,www.aier.org
5,http://rte.ie/news/world/2020/1012/1171035-car...,Teenager one step from becoming first millenni...,2020-10-12T15:21:15.903000+01:00,[],A British-born Italian teenager who dedicated ...,http://rte.ie,RTE.ie,False,False,newspaperbasic,2020-10-12T22:03:12.218333+00:00,NaN,rte.ie
6,https://www.inforum.com/newsmd/coronavirus/671...,Minnesota to require low-risk care homes be re...,None,"[Written, Paul John Scott, Oct -]","ROCHESTER, Minn. — Stressing the need to balan...",https://www.inforum.com,INFORUM,False,False,newspaperbasic,2020-10-12T22:03:12.263381+00:00,NaN,NaN
7,https://www.postbulletin.com/news/crime-and-co...,Rochester Police Department daily incident rep...,None,"[Written, Andrew Link, Oct -]",Wondering what the commotion was in your neigh...,https://www.postbulletin.com,Post Bulletin,False,False,newspaperbasic,2020-10-12T22:03:12.301717+00:00,NaN,NaN
8,http://refinery29.com/de-de/2020/10/10080798/g...,„Gilmore Girls“: 16 berühmte Nebendarsteller*i...,2020-10-10T00:00:00,"[Lindsay Denninger, Von Ineye Komonibo, Ineye ...",Ich spreche hier also nicht von Melissa McCart...,http://refinery29.com,,False,False,newspaperbasic,2020-10-12T22:03:13.774399+00:00,NaN,NaN
9,http://elnuevodia.com/noticias/mundo/videos/nu...,Nuevo misil balístico de Corea del Norte ¿Un d...,None,[El Nuevo Día],,http://elnuevodia.com,El Nuevo Día,False,False,newspaperbasic,2020-10-12T22:03:13.792088+00:00,NaN,NaN


In [100]:
df_src_norm_limited['flags.in_graph'].value_counts()

False    35878
True     24496
Name: flags.in_graph, dtype: int64

In [124]:
df_src_norm_limited.shape

(60374, 13)

In [126]:
!pip install furl

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [150]:
# strip url of www and http

from furl import furl
def strip_url(url):
    #if "youtube.com/watch" in url:
    #    f = furl(url)
    #    try:
    #        v = f.args['v']
    #        url = f.remove(args=True, fragment=True).url
    #        url = furl(url).add(args={'v':v}).url
    #    except:
    #        pass
    #else:
    #    url = furl(url).remove(args=True, fragment=True).url
    
    if '://www.' in url:
        url = url.split('://www.',1)[1]
    elif '://' in url:
        url = url.split('://',1)[1]
    return url

In [151]:
strip_url('http://actionnewsjax.com')

'actionnewsjax.com'

In [157]:
# adding a new column domain_url by stripping source.url using strip_url function
df_src_norm_limited['domain_url']= df_src_norm_limited['source.url'].apply(strip_url)

In [156]:
df_src_norm_limited.head(10)

,url,extracted.title,extracted.date,extracted.authors,extracted.text,source.url,source.sitename,flags.in_graph,flags.has_nlp,context.scraper,context.datetime,flags.skipped_graph,metadata.dump.twitter.domain,domain_url
0,http://actionnewsjax.com/news/trending/amy-con...,Amy Coney Barrett hearing Day 2: Questioning b...,None,"[Debbie Lord, Cox Media Group National Content...",“His judicial philosophy was straightforward: ...,http://actionnewsjax.com,WJAX,False,False,newspaperbasic,2020-10-12T22:03:11.021791+00:00,NaN,NaN,actionnewsjax.com
1,https://www.13newsnow.com/article/news/nation-...,"Vanessa Bryant wishes Kobe, Gigi could've seen...",2020-10-12T19:24:52,"[Author, Douglas Jones, Tegna]",Vanessa Bryant wrote that she wished Kobe and ...,https://www.13newsnow.com,13newsnow.com,False,False,newspaperbasic,2020-10-12T22:03:11.185397+00:00,NaN,www.13newsnow.com,13newsnow.com
2,http://digg.com/digg-picks/link/kindles-video-...,"Kindles, Video Doorbells And More Are On Sale ...",None,[Digg Editors],👋 Welcome to Digg\n\nThanks for creating an ac...,http://digg.com,,False,False,newspaperbasic,2020-10-12T22:03:11.585230+00:00,NaN,NaN,digg.com
3,http://sports.mynorthwest.com/1199798/clayton-...,Seahawks Observations: John Clayton on 5-0 Haw...,2020-10-12T21:52:23+00:00,"[John Clayton, Share, October, At Pm]",The Seahawks’ 27-26 victory over the Minnesota...,http://sports.mynorthwest.com,sports.MyNorthwest.com,False,False,newspaperbasic,2020-10-12T22:03:12.004826+00:00,NaN,NaN,sports.mynorthwest.com
4,https://www.aier.org/article/lockdowns-have-ki...,Lockdowns Have Killed What’s Left of the Unite...,2020-10-12T12:30:15+00:00,"[Peter C. Earle, October, Peter C. Earle Is An...",,https://www.aier.org,AIER,False,False,newspaperbasic,2020-10-12T22:03:12.084973+00:00,NaN,www.aier.org,aier.org
5,http://rte.ie/news/world/2020/1012/1171035-car...,Teenager one step from becoming first millenni...,2020-10-12T15:21:15.903000+01:00,[],A British-born Italian teenager who dedicated ...,http://rte.ie,RTE.ie,False,False,newspaperbasic,2020-10-12T22:03:12.218333+00:00,NaN,rte.ie,rte.ie
6,https://www.inforum.com/newsmd/coronavirus/671...,Minnesota to require low-risk care homes be re...,None,"[Written, Paul John Scott, Oct -]","ROCHESTER, Minn. — Stressing the need to balan...",https://www.inforum.com,INFORUM,False,False,newspaperbasic,2020-10-12T22:03:12.263381+00:00,NaN,NaN,inforum.com
7,https://www.postbulletin.com/news/crime-and-co...,Rochester Police Department daily incident rep...,None,"[Written, Andrew Link, Oct -]",Wondering what the commotion was in your neigh...,https://www.postbulletin.com,Post Bulletin,False,False,newspaperbasic,2020-10-12T22:03:12.301717+00:00,NaN,NaN,postbulletin.com
8,http://refinery29.com/de-de/2020/10/10080798/g...,„Gilmore Girls“: 16 berühmte Nebendarsteller*i...,2020-10-10T00:00:00,"[Lindsay Denninger, Von Ineye Komonibo, Ineye ...",Ich spreche hier also nicht von Melissa McCart...,http://refinery29.com,,False,False,newspaperbasic,2020-10-12T22:03:13.774399+00:00,NaN,NaN,refinery29.com
9,http://elnuevodia.com/noticias/mundo/videos/nu...,Nuevo misil balístico de Corea del Norte ¿Un d...,None,[El Nuevo Día],,http://elnuevodia.com,El Nuevo Día,False,False,newspaperbasic,2020-10-12T22:03:13.792088+00:00,NaN,NaN,elnuevodia.com


In [160]:
#Now, join source-lean-flags dataframe with the main dataframe

dfinal = df_src_norm_limited.merge(df_src_flags, how='inner', left_on='domain_url', right_on='domain')

In [162]:
dfinal.head(10)

,url,extracted.title,extracted.date,extracted.authors,extracted.text,source.url,source.sitename,flags.in_graph,flags.has_nlp,context.scraper,...,flags.skipped_graph,metadata.dump.twitter.domain,domain_url,domain,bias_score,factually_questionable_flag,propaganda_flag,conspiracy_flag,hate_group_flag,satire_flag
0,http://actionnewsjax.com/news/trending/amy-con...,Amy Coney Barrett hearing Day 2: Questioning b...,None,"[Debbie Lord, Cox Media Group National Content...",“His judicial philosophy was straightforward: ...,http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,NaN,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
1,http://actionnewsjax.com/news/trending/john-mc...,"John McCain’s mother, Roberta McCain, dies at 108",None,"[Theresa Seiger, Cox Media Group National Cont...","Known for her feisty and energetic spirit, Rob...",http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,NaN,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
2,http://actionnewsjax.com/news/local/duval-coun...,Police: Business owner shoots burglary suspect...,None,"[Actionnewsjax.Com News Staff, Bridgette Matte...",“He was hiding under all these and bleeding ri...,http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,NaN,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
3,http://actionnewsjax.com/news/politics/biden-m...,"Biden makes big push in Ohio, once seen as lon...",None,"[Bill Barrow, Will Weissert, Updated, October,...",The Biden campaign has increased advertising i...,http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,NaN,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
4,http://actionnewsjax.com/news/disney-reorg-fur...,Disney reorg to further bolster company's focu...,None,"[Tali Arbel, Updated, October, - Pm, Kelli Dug...",The coronavirus pandemic has hit box-office re...,http://actionnewsjax.com,WJAX,True,False,newspaperbasic,...,NaN,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
5,http://actionnewsjax.com/news/politics/barrett...,"Barrett vows fair approach as justice, Democra...",None,"[Mark Sherman, Lisa Mascaro, Mary Clare Jaloni...",Testifying from her office because of the pand...,http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,True,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
6,http://actionnewsjax.com/news/local/duval-coun...,As millions of Americans get ready for Amazon ...,None,"[Dani Bozzini, Action News Jax, Updated, Octob...","When the pandemic began, Native Sun shifted it...",http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,True,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
7,http://actionnewsjax.com/news/local/clay-count...,Clay County community gathers to honor lives o...,None,"[Ryan Nelson, Action News Jax, Updated, Octobe...","Rowley died at the scene, and Kennedy died at ...",http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,NaN,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
8,http://actionnewsjax.com/news/chinas-trade-gro...,China's trade growth accelerates in Sept; expo...,None,"[Joe Mcdonald, Updated, October, - Am, Michell...",China became the first major economy to reboun...,http://actionnewsjax.com,WJAX,False,False,newspaperbasic,...,True,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0
9,http://actionnewsjax.com/news/global-shares-mo...,Global shares mostly lower as virus concerns t...,None,"[Elaine Kurtenbach, Updated, October, - Am, Mi...",Optimism over hopes for new aid for the U.S. e...,http://actionnewsjax.com,WJAX,True,False,newspaperbasic,...,NaN,NaN,actionnewsjax.com,actionnewsjax.com,0.0,0,0,0,0,0


In [163]:
dfinal.shape

(51102, 21)

In [51]:
#df_src_norm_limited=df_src_norm.filter(items=['url','source.sitename','extracted.title', 'extracted.date', 'extracted.text', 'metadata.keywords', 'metadata.description'])

In [58]:
#df_src_norm_limited.shape

In [59]:
#df_src_norm_limited.head(10)

In [60]:
#df_src_norm_limited['metadata.description'][0]

In [61]:
#df_src_norm_limited['extracted.text'][0]

In [ ]:
#df_src_norm_limited['url'][0]

In [ ]:
#df_src_norm_limited.describe

## Create Datasets for further analysis


In [62]:
#extracted_text=df_src_norm_limited['extracted.text']
#extracted_text.dropna()


In [63]:
#extracted_text.shape

# Explore some transformers

### https://github.com/UKPLab/sentence-transformers

In [17]:
!pip install -U sentence-transformers

     |████████████████████████████████| 59 kB 3.6 MB/s eta 0:00:011
     |████████████████████████████████| 1.1 MB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 35.6 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.7.2-py3-none-any.whl size=91186 sha256=5311dc98f80cdddd2f83dd7f4a5f22db20c5f542aa9ddc5705bfc13d36e6977b
  Stored in directory: /root/.cache/pip/wheels/df/42/15/b8329fd622ed2e9ee15b00314a387aa7d70e402d144349d540
Successfully built sentence-transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0
  Attempting uninstall: transformers
    Found existing installation: transformers 2.10.0
    Uninstalling transformers-2.10.0:
      Successfully uninstalled transformers-2.10.0
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [66]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
sentence_embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, sentence_embeddings):
    print("Sentence:", sentence)
    #print("Embedding:", embedding)
    print("")

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Sentence: This framework generates embeddings for each input sentence

Sentence: Sentences are passed as a list of string.

Sentence: The quick brown fox jumps over the lazy dog.



In [64]:
#extracted_text.head(10)

In [24]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

sentences = ['A man is eating food.',
          'A man is eating a piece of bread.',
          'The girl is carrying a baby.',
          'A man is riding a horse.',
          'A woman is playing violin.',
          'Two men pushed carts through the woods.',
          'A man is riding a white horse on an enclosed ground.',
          'A monkey is playing drums.',
          'Someone in a gorilla costume is playing a set of drums.'
          ]

#Encode all sentences
embeddings = model.encode(sentences)

#Compute cosine similarity between all pairs
cos_sim = util.pytorch_cos_sim(embeddings, embeddings)

#Add all pairs to a list with their cosine similarity score
all_sentence_combinations = []
for i in range(len(cos_sim)-1):
    for j in range(i+1, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

#Sort list by the highest cosine similarity score
all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)

print("Top-5 most similar pairs:")
for score, i, j in all_sentence_combinations[0:5]:
    print("{} \t {} \t {:.4f}".format(sentences[i], sentences[j], cos_sim[i][j]))

Top-5 most similar pairs:
A man is eating food. 	 A man is eating a piece of bread. 	 0.7210
A monkey is playing drums. 	 Someone in a gorilla costume is playing a set of drums. 	 0.6435
A man is riding a horse. 	 A man is riding a white horse on an enclosed ground. 	 0.6343
A man is eating food. 	 A man is riding a horse. 	 0.2840
A man is eating a piece of bread. 	 A monkey is playing drums. 	 0.2451


In [65]:
#metadata_description=df_src_norm_limited['metadata.description']